In [5]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key =openai_api_key

In [ ]:
import csv
import json

# CSV 파일 경로
csv_file = "../관광지-1.csv"
json_output = []

# CSV 파일 읽기
with open(csv_file, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) == 3:  # KO, ENG, JPN 데이터가 있는 경우만 처리
            ko, eng, jpn = row
            json_entry = {"KO": ko, "ENG": eng, "JPN": jpn}
            json_output.append(json_entry)

# 데이터프레임 생성
df = pd.DataFrame({"metadata": [json.dumps(entry, ensure_ascii=False) for entry in json_output]})

# 엑셀 파일 저장
df.to_excel("hyun_places.xlsx", index=False)

In [ ]:
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain

def generate_sentences_from_csv(df, openai_api_key):
    llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
    results = []
    
    # Korean sentence generation prompt
    korean_prompt = PromptTemplate(
        input_variables=["word"],
        template="""
        You are a professional Korean language educator.
        Create a natural Korean sentence that includes the word '{word}'.
        Provide only the sentence without the original sentence, additional examples, symbols, or punctuation explanations.
        """
    )
    
    # English translation prompt
    english_prompt = PromptTemplate(
        input_variables=["hint", "korean_sentence"],
        template="""
        You are a professional English translator.
        Translate the following Korean sentence into English using the word '{hint}': {korean_sentence}.
        Provide only the translated text without the original sentence, additional examples, symbols, or punctuation explanations.
        """
    )
    
    # Create individual chains
    korean_chain = LLMChain(
        llm=llm,
        prompt=korean_prompt,
        output_key="korean_sentence"
    )
    
    english_chain = LLMChain(
        llm=llm,
        prompt=english_prompt,
        output_key="english_translation"
    )
    
    # Combine chains
    translation_chain = SequentialChain(
        chains=[korean_chain, english_chain],
        input_variables=["word", "hint"],
        output_variables=["korean_sentence", "english_translation"],
    )
    
    for index, row in df.iterrows():
        metadata_dict = json.loads(row["metadata"])
        
        # Execute the sequential chain
        chain_output = translation_chain({
            "word": metadata_dict["KO"],
            "hint": metadata_dict["ENG"]
        })
        
        # Prepare results
        results.append([
            "Translate the Korean into English. Refer to the glossary, and if it is not in the glossary, translate it. Do not provide explanations.",
            chain_output["korean_sentence"],
            chain_output["english_translation"],
            "ENG",
            row["metadata"]
        ])
    
    # Create and save DataFrame
    result_df = pd.DataFrame(
        results,
        columns=["instruction", "input", "output", "language", "metadata"]
    )
    result_df.to_excel("bbbb.xlsx", index=False)
    
    print("저장됨_한일")

# Load and process the Excel file
generate_sentences_from_csv(df, openai_api_key)

C:\Users\201\AppData\Local\Temp\ipykernel_18640\1881773310.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
C:\Users\201\AppData\Local\Temp\ipykernel_18640\1881773310.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  korean_chain = LLMChain(
C:\Users\201\AppData\Local\Temp\ipykernel_18640\1881773310.py:55: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain_output = translation_ch